In [ ]:
import bnpm

bnpm.image_processing.prepare_cv2_imshow()

import ptlflow

import torch

torch.__version__

import decord

with open(
    '/media/rich/bigSSD/other lab data/Svoboda_lab/BCI34_2022-07-19/side/2022-07-19_13-34-06/trial_00000__2022-07-19_13-34-07.mp4',
    'rb'
) as f:
    vr = decord.VideoReader(f, ctx=decord.cpu(0))

import matplotlib.pyplot as plt

plt.figure()
plt.imshow(vr[950].asnumpy())

img1 = vr[940].asnumpy()
img2 = vr[950].asnumpy()

# Additional dependencies for this example
from ptlflow.utils.io_adapter import IOAdapter
from ptlflow.utils import flow_utils

import numpy as np
import cv2
from tqdm import tqdm
import bnpm

DEVICE = 'cuda:0'

# Get an initialized model from PTLFlow
model = ptlflow.get_model('fastflownet', 'things')
# model = ptlflow.get_model('liteflownet2',)
model.eval();

model.to(DEVICE);

# IOAdapter is a helper to transform the two images into the input format accepted by PTLFlow models
io_adapter = IOAdapter(model, img1.shape[:2])
img_array = np.stack([img1, img2, ], axis=0)
inputs = io_adapter.prepare_inputs(img_array)

frames = io_adapter.prepare_inputs(vr[:2000].asnumpy())
frames_device = {'images': frames['images'].to(DEVICE)}

predictions = []
for ii in tqdm(range(frames_device['images'].shape[1])):
    if ii>0:
        two_frames = {'images': frames_device['images'][:,ii-1:ii+1,...]}
        flows = model(two_frames)['flows'].detach().cpu()[0,0,...]
        predictions.append(flows)
#     print(ii)
#     1-1
    

# Visualize the predicted flow
# flow = predictions['flows'][0, 0]  # Remove batch and sequence dimensions
flow = torch.stack(predictions, dim=0)
flow = flow.permute(0,2,3,1)  # change from CHW to HWC shape
flow = flow.detach().numpy()
# flow_viz = flow_utils.flow_to_rgb(flow)  # Represent the flow as RGB colors
# flow_viz = cv2.cvtColor(flow_viz, cv2.COLOR_BGR2RGB)  # OpenCV uses BGR format
# plt.imshow(flow_viz)

import torchvision

del model

del frames_device

import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()
gc.collect()

flow_viz = torchvision.utils.flow_to_image(torch.clip(torch.stack(predictions, dim=0).to('cpu'), min=-3, max=3)).cpu()

bnpm.image_processing.play_video_cv2(flow_viz.permute(0,2,3,1).numpy(), frameRate=100)

bnpm.image_processing.play_video_cv2(vr[:2000].asnumpy(), frameRate=200)